In [8]:
from tkinter import *
from PIL import ImageTk, Image
from datetime import datetime
from tkinter import filedialog
import os
import cv2

'''This python class manages the GUI of our cutCAPTCHA solver, it handles all the buttons of the user
interface along with their command to execute'''
class MySolverGUI():
    #The class constructor invoke each time a new instance of the class is created
    def __init__(self, root):
        # Variables used by the class
        self.root = root
        self.remaining = 0
        self.welcometext="Welcome to cutcaptcha solver !"
        self.errorimg="Cutcaptcha images missing or do not exist in the folder !!!"
        self.errorfolder="Please select a valid folder !!!"
        self.timertext="Timer: "
        self.executiontext="Solving time: "
        self.key=""
        self.question=""
        # Windows title
        self.root.title('CUTCAPTCHA SOLVER')
        # Windows geometry
        self.root.geometry('{}x{}'.format(800, 600))
        # Create a menubar
        self.menubar = Menu(self.root)
        self.root.config(menu=self.menubar)
        # Create the file_menu
        self.file_menu = Menu(self.menubar, tearoff=0)
        # Add Open... menu to the File menu
        self.file_menu.add_command(label='Open...', command=self.openCaptcha)
        # Add the separator to the file menu
        self.file_menu.add_separator()
        # Add Exit menu item
        self.file_menu.add_command(label='Exit', command=self.root.destroy)
        # Add the File menu to the menubar
        self.menubar.add_cascade(label="File", menu=self.file_menu)
        
        # create all of the main containers
        # Top container with cyan background
        self.top_frame = Frame(self.root, bg='cyan', width=450, height=50, pady=3)
        # Center container with grey background
        self.center = Frame(self.root, bg='grey', width=50, height=40, padx=3, pady=3)
        # Bottom container with lavender background
        self.btm_frame = Frame(self.root, bg='lavender', width=450, height=60, pady=3)
        
        # layout all of the main containers
        self.root.grid_rowconfigure(1, weight=1)
        self.root.grid_columnconfigure(0, weight=1)
        # Set up the main containers in a grid of 3 rows and 1 column, each container per row
        self.top_frame.grid(row=0, sticky="ew")
        self.center.grid(row=1, sticky="nsew")
        self.btm_frame.grid(row=3, sticky="ew")

        # layout the widgets in the top frame
        self.labels = Frame(self.top_frame, bg='cyan')
        self.labels.pack()
        # Label for the timer
        self.label_timer=Label(self.labels, text=''+self.timertext,font=('bold',20), background="cyan")
         # Label for the solution
        self.label_sol=Label(self.labels, text=''+self.welcometext,font=('bold',20), background="cyan")
        # set the timer label on top and the solution label on bottom
        self.label_timer.pack(side = 'top')
        self.label_sol.pack(side = 'bottom')
        # create the center widgets that will contain the challenge main image and the images of the puzzle pieces
        self.center.grid_rowconfigure(1, weight=1)
        self.center.grid_columnconfigure(0, weight=1)
        # The main image and the puzzle image are organised in a Grid of 2 rows and 1 column 
        # the main image widgets is on top whereas the widget of the puzzle piece images are on bottom
        self.parts_frame = Frame(self.center, bg='grey' ,width=488, height=100)
        self.imgs_frame = Frame(self.center, bg='lavender',width=488, height=332, padx=5, pady=5)
        self.imgs_frame.grid(row=0, column=0)
        self.parts_frame.grid(row=1, column=0)


        self.part0_frame = Frame(self.parts_frame, bg='grey', padx=5)
        self.part1_frame = Frame(self.parts_frame, bg='grey', padx=5)
        self.part2_frame = Frame(self.parts_frame, bg='grey', padx=5)
        self.part0_frame.grid(row=0, column=0)
        self.part1_frame.grid(row=0, column=1)
        self.part2_frame.grid(row=0, column=2)
        
        #No images for the challenge at the begining
        self.noImg=ImageTk.PhotoImage(Image.open("./cutcaptchas/no-image.png"))
        #self.main_label = Label(self.imgs_frame, image='')
        self.main_label = Label(self.imgs_frame, image=self.noImg)
        self.main_label.pack()
        # No image also for the puzzle pieces at the begining
        self.noPart=ImageTk.PhotoImage(Image.open("./cutcaptchas/no-part.png"))

        self.part0_label = Label(self.part0_frame, image=self.noPart)
        self.part1_label = Label(self.part1_frame, image=self.noPart)
        self.part2_label = Label(self.part2_frame, image=self.noPart)
        self.part0_label.grid(row=0, column=0)
        self.part1_label.grid(row=0, column=1)
        self.part2_label.grid(row=0, column=2)


        # create the widgets for the bottom frame that will contains the buttons ('New' and 'Solve') of our application
        self.buttons = Frame(self.btm_frame, bg='lavender')
        self.buttons.pack()

        self.bt_new=Button(self.buttons, text='New', command=self.newChallenge)
        self.bt_solve=Button(self.buttons, text='Solve', command=self.solve)
        self.bt_new.pack(side = 'left')
        self.bt_solve.pack(side = 'right')
        #hide the label
        self.label_timer.pack_forget()
        #hide the button 'solve'
        self.bt_solve.pack_forget()
    
    # Function that will handle the coutdown on the application before downloading the images
    def countdown(self, remaining = None):
        if remaining is not None:
            self.remaining = remaining
        # While the time remaining in the countdown is not 0 show the timer label with the remaining time    
        if self.remaining <= 0:
            self.label_timer.pack_forget()
        else:
            self.label_timer.pack()
            self.label_timer.configure(text=self.timertext+"%d" % self.remaining)
            self.remaining = self.remaining - 1
            self.root.after(1000, self.countdown)
    
    # Funtion that request a new key to the cutCAPTCHA server for a new challenge          
    def newChallenge(self):
        self.label_sol.pack_forget()
        # We request a key from the server through a static method of another class named MySolver
        key, time=MySolver.requestCaptcha()
        self.key=key
        # Clear all the images
        self.clearImages()
        # Start the countdown once the waiting time is obtained from the server
        self.countdown(time)
        self.root.after(time*1000, self.getImgs)
    
    #Function that request the Id of a CAPTCHA from the server and download the images in the corresponding frames
    def getImgs(self):
        # Each time we download new images we need to reset the text of the solution label with the welcome text
        self.label_sol.configure(text=''+self.welcometext)
        self.label_sol.pack()
        # We request a captcha ID (using the key obtained previously) from the server through a static method of another class named MySolver
        token, question=MySolver.requestQuestion(self.key)
        self.question=question
        # We download the challenge images (using the captcha ID obtained previously) from the server through a static method of another class named MySolver
        MySolver.downloadCaptcha(question)
        #After downloading the image show them in their respective frames
        self.loadImages()
        
    # Function that finds the solution of a captcha challenge
    def solve(self):
        # We save the start time in order to evaluate the solving time
        start_time = datetime.now()
        # We find the solution of the challenge (requested previsouly from the server) through a static method of another class named MySolver
        X0,Y0,X1,Y1,X2,Y2=MySolver.solveCaptcha()
        # We save the end time in order to evaluate the solving time
        end_time = datetime.now()
        # we evaluate the solving time
        time_diff = (end_time - start_time)
        execution_time = time_diff.total_seconds() * 1000
        # We update the timer label with a text printing out the solving time
        self.label_timer.configure(text=self.executiontext+"%.3f ms" % execution_time)
        self.label_timer.pack()
        # We also update the frame of the main image of the challenge with the image recovered with the solution
        self.main = ImageTk.PhotoImage(Image.open(MySolver.path+"captchasolved.png"))
        self.main_label.configure(image=self.main)
        # We hide the puzzle pieces image as the main image shows already the recovered image 
        self.part0_label.grid_forget()
        self.part1_label.grid_forget()
        self.part2_label.grid_forget()
        # We update the solution label with the final solution (as (x,y) coordinates for the right position of each puzzle piece)
        self.label_sol.configure(text="Solution: [(%d, %d), (%d, %d), (%d, %d)]" % (X0,Y0,X1,Y1,X2,Y2), fg='black')
        self.label_sol.pack()
        self.bt_solve.pack_forget()
        
    # Function that clears all the challenges images      
    def clearImages(self):
        self.main_label.configure(image=self.noImg)
        self.main_label.pack()
        self.part0_label.configure(image=self.noPart)
        self.part1_label.configure(image=self.noPart)
        self.part2_label.configure(image=self.noPart)
        self.part0_label.grid()
        self.part1_label.grid()
        self.part2_label.grid()
        
    # Function that loads all the challenges images     
    def loadImages(self):
        #Hide the label timer
        self.label_timer.pack_forget()
        # If the challenge images have been downloaded properly
        if os.path.exists(MySolver.path+"cut.png") and os.path.exists(MySolver.path+"part0.png") and os.path.exists(MySolver.path+"part1.png") and os.path.exists(MySolver.path+"part2.png"):
            #if cutcaptcha.png already exist then it will become the new cut.png
            if os.path.exists(MySolver.path+"cutcaptcha.png"):
                bck = cv2.imread(""+MySolver.path+"cutcaptcha.png")
                cv2.imwrite(""+MySolver.path+"cut.png",bck)
            #Show welcome text
            self.label_sol.configure(text=''+self.welcometext, fg='black')
            self.label_sol.pack()
            # Place each challenge image in its corresponding frame
            self.main = ImageTk.PhotoImage(Image.open(MySolver.path+"cut.png"))
            self.main_label.configure(image=self.main)
            self.main_label.pack()
            self.part0=ImageTk.PhotoImage(Image.open(MySolver.path+"part0.png"))
            self.part1=ImageTk.PhotoImage(Image.open(MySolver.path+"part1.png"))
            self.part2=ImageTk.PhotoImage(Image.open(MySolver.path+"part2.png"))
            self.part0_label.configure(image=self.part0)
            self.part1_label.configure(image=self.part1)
            self.part2_label.configure(image=self.part2)
            self.part0_label.grid()
            self.part1_label.grid()
            self.part2_label.grid()
            #show the button 'solve'
            self.bt_solve.pack()
        # If the challenges images have not been downloaded properly, show the error message    
        else:
            #Show error text on the images
            self.label_sol.configure(text=''+self.errorimg, fg='red')
            self.label_sol.pack()
            #Clear all the images
            self.clearImages()
            
        
    # Function that handle the filedialog for the working folder of the captcha solver    
    def openCaptcha(self):
            # we call the filedialog function of python to pop up a file dialog to the user so that he can select the working folder for the captcha solver
            folder_selected = filedialog.askdirectory()
            # If the user has selected the a valide folder
            if (len(folder_selected) != 0):
                MySolver.changePath(folder_selected+'/')
                self.loadImages()
            #if the user cancel the choice or he folder has not been selected...
            else:
                #Show error text on the folder
                self.label_sol.configure(text=''+self.errorfolder, fg='red')
                self.label_sol.pack()
                #Clear all the images
                self.clearImages()


In [9]:
import requests
from requests.structures import CaseInsensitiveDict
from bs4 import BeautifulSoup as bs
from time import sleep, time
from PIL import Image
import json
import warnings
import numpy as np
import cv2
import os

warnings.filterwarnings('ignore')

'''This python class manages the main functions of our cutCAPTCHA solver, it handles all
the functions used by us to solve a cutcaptcha challenge'''
class MySolver():
    
    oldpath="./cutcaptchas/"
    
    path=oldpath
    
    def __init__(self, path):
        MySolver.path = path
        
    @staticmethod
    def changePath(newpath):
        MySolver.path = newpath
    
    # Function that return the area (as box) within an image that does not contains white pixels
    @staticmethod
    def bbox(im):
        a = np.array(im)[:,:,:3]  # keep RGB only
        m = np.any(a != [255, 255, 255], axis=2)
        coords = np.argwhere(m)
        y0, x0, y1, x1 = *np.min(coords, axis=0), *np.max(coords, axis=0)
        return (x0, y0, x1+1, y1+1)
    
    # Function that extract the area (as box) within an image that does not contains white pixels
    # and sets a transparent background to the resulted image
    @staticmethod
    def removeBackground(image):
        png = Image.open(""+MySolver.path+""+image).convert('RGBA')
        background = Image.new('RGBA', png.size, (255,255,255,0))
        alpha_composite = Image.alpha_composite(background, png)
        im2 = alpha_composite.crop(MySolver.bbox(alpha_composite))
        # We save the processed image on the disk with 'X_' as prefix
        im2.save(''+MySolver.path+'X_'+image, 'PNG', quality=100)
    
    # Function that detect the edge of the challenge main image using canny algorithm
    @staticmethod
    def edgeMainDetect(image):
        img = cv2.imread(""+MySolver.path+""+image) # Read image
        mask = np.zeros(img.shape[:2],np.uint8)
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)
        rect = (10,10,470,320)
        cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        img = img*mask2[:,:,np.newaxis]
        # we erase the text cutcptcha from the image
        cv2.rectangle(img, (0,300), (175,332), (0,0,0), -1)
        #edge = cv2.Canny(img, 400,400)
        # We apply the canny algorithm to detect the edge
        edge = cv2.Canny(img, 200,200)
        # We save the processed image on the disk with 'EDGE_' as prefix
        cv2.imwrite(""+MySolver.path+"EDGE_"+image,edge)
        
    # Function that detect the edge of the puzzle piece images using canny algorithm
    @staticmethod
    def edgePartDetect(image):
        MySolver.removeBackground(""+image)
        img = cv2.imread(""+MySolver.path+"X_"+image) # Read image
        imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 255, 255, 255)
        thresh = cv2.bitwise_not(thresh)
        # We determine the contours (shape) of the puzzle piece image
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #if the shape is opened we draw the contours to close it else we applied directly the edge detection
        opened=True
        for i in range(len(contours)):
            if (hierarchy[0][i][2]<0 and hierarchy[0][i][3]<0):
                opened=False
        if(opened==True):
            res=cv2.drawContours(img, contours, -1, (0,255,0), 0)
        else:
            res=img
        #edge = cv2.Canny(res, 200,200)
        # We apply the canny algorithm to detect the edge
        edge = cv2.Canny(res, 500,500)
        # We save the processed image on the disk with 'EDGE_' as prefix
        cv2.imwrite(""+MySolver.path+"EDGE_"+image,edge)

    # Function that detect the position of the edge of a puzzle piece image within
    # the edge of the challenge main image (using template matching method of openCV)
    @staticmethod
    def positionDetect(big, small):
        method = cv2.TM_SQDIFF
        # Read the image edges from the files
        small_image = cv2.imread(''+MySolver.path+'EDGE_'+small)
        large_image = cv2.imread(''+MySolver.path+'EDGE_'+big)
        tmplt_mask = cv2.imread(''+MySolver.path+'EDGE_'+small, cv2.COLOR_BGR2GRAY)
        # We apply template matching of OpenCV to get the right position of the puzzle piece
        result = cv2.matchTemplate(large_image, small_image, method, mask=tmplt_mask)
        # We want the minimum squared difference
        mn,_,mnLoc,_ = cv2.minMaxLoc(result)
        # Extract the coordinates of our best match
        MPx,MPy = mnLoc
        #print(MPx,MPy)
        return MPx,MPy
    
    # Function that determines the coordinates of the non-white pixel within an image containing white 
    #and non-white pixels
    @staticmethod
    def coordinatesImg(image):
        png = Image.open(""+MySolver.path+""+image).convert('RGBA')
        background = Image.new('RGBA', png.size, (255,255,255,0))
        alpha_composite = Image.alpha_composite(background, png)
        x0, y0, x1, y1 =MySolver.bbox(alpha_composite)
        #print(x0, y0, x1, y1)
        return x0, y0, x1, y1
    
    #Function that return the size of an image (Widht times Height)
    @staticmethod
    def getSize(image):
        width, height = Image.open(''+MySolver.path+'X_'+image).size
        return width*height
    
    # Function that request a key and the waiting time from the cutCAPTCHA server for a challenge
    @staticmethod
    def requestCaptcha():
        r = requests.get('https://v2.cutcaptcha.net/captcha/SAs61IAI.json', verify=False) 
        #print(r.text)
        #sleep(2)
        # We get the JSON data from the response of the server
        json_data = json.loads(r.text)
        while('timer' not in json_data):
            return MySolver.requestCaptcha()
        # We extract the time and the key from the JSON data
        key=json_data['timer']['key']
        time=json_data['timer']['time']
        #print(key, time)
        return key, time
    
    # Function that request a captcha ID from the cutCAPTCHA server for a challenge using a key obtained from the server
    @staticmethod
    def requestQuestion(key):
        url = "https://v2.cutcaptcha.net/captcha/SAs61IAI.json"
        headers = CaseInsensitiveDict()
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        data = "api_key=SAs61IAI&tk="+key
        resp = requests.post(url, headers=headers, data=data, verify=False)
        #print(resp.text)
        json_data = json.loads(resp.text)
        while('captcha_question' not in json_data):
            return MySolver.requestQuestion(key)
        token=json_data['captcha_token']
        question=json_data['captcha_question']
        #print(token,question)
        return token, question
    
    # Function that download the challenge images rom the cutCAPTCHA server and save them on the disk using a captcha ID obtained from the server
    @staticmethod
    def downloadCaptcha(question):
        MySolver.path=MySolver.oldpath
        newpath=MySolver.path+''+question+'/'
        
        if not os.path.exists(''+newpath):
            os.makedirs(''+newpath)
            
        MySolver.path=newpath
        
        # Request to download the challenge main image
        image_main_url ="https://v2.cutcaptcha.net/captcha/SAs61IAI/"+question+"/cut.png"
        image_main_data = requests.get(image_main_url, verify=False).content 
        
        # Request to download the image of puzzle piece number 1
        image_part0_url ="https://v2.cutcaptcha.net/captcha/SAs61IAI/"+question+"/part0.png"
        image_part0_data = requests.get(image_part0_url, verify=False).content 

        # Request to download the image of puzzle piece number 2
        image_part1_url ="https://v2.cutcaptcha.net/captcha/SAs61IAI/"+question+"/part1.png"
        image_part1_data = requests.get(image_part1_url, verify=False).content 
        
        # Request to download the image of puzzle piece number 3
        image_part2_url ="https://v2.cutcaptcha.net/captcha/SAs61IAI/"+question+"/part2.png"
        image_part2_data = requests.get(image_part2_url, verify=False).content 
        
        # Save the challenge main image downloaded on the disk
        with open(''+MySolver.path+'cut.png', 'wb') as handler: 
               handler.write(image_main_data)
        # Save the image of puzzle piece number 1 downloaded on the disk
        with open(''+MySolver.path+'part0.png', 'wb') as handler: 
               handler.write(image_part0_data)
        
        # Save the image of puzzle piece number 2 downloaded on the disk
        with open(''+MySolver.path+'part1.png', 'wb') as handler: 
               handler.write(image_part1_data)
                
        # Save the image of puzzle piece number 3 downloaded on the disk
        with open(''+MySolver.path+'part2.png', 'wb') as handler: 
               handler.write(image_part2_data)
    
    # Function that returned the solution of a cutCAPTCHA challenge downloaded
    @staticmethod
    def solveCaptcha():
        found0=False
        found1=False
        found2=False
        main_img='cut.png'
        part0_img='part0.png'
        part1_img='part1.png'
        part2_img='part2.png'
        solx0=0
        soly0=0
        solx1=0
        soly1=0
        solx2=0
        soly1=0
        lst=[0,0,0]
        #backup the main challenge image
        img = cv2.imread(""+MySolver.path+"cut.png")
        cv2.imwrite(""+MySolver.path+"cutcaptcha.png",img)
        # We apply the edge detection function on each puzzle image
        MySolver.edgePartDetect(''+part0_img)
        MySolver.edgePartDetect(''+part1_img)
        MySolver.edgePartDetect(''+part2_img)
        # We determine the sizes of the puzzle piece images as a list
        lst[0]=MySolver.getSize(''+part0_img)
        lst[1]=MySolver.getSize(''+part1_img)
        lst[2]=MySolver.getSize(''+part2_img)
        # While all the positions are not found
        while(found0==False or found1==False or found2==False):
            # Apply the edge detection function on the main image
            MySolver.edgeMainDetect(''+main_img)
            # If a puzzle position has not been found yet we try to found by calling the function to detect the position of the pieces using template matching on edge images
            if(found0==False):
                 solx0,soly0=MySolver.positionDetect(main_img,part0_img)
            if(found1==False):
                solx1,soly1=MySolver.positionDetect(main_img,part1_img)
            if(found2==False):
                solx2,soly2=MySolver.positionDetect(main_img,part2_img)
            # We first start placing the puzzle pieces with the biggest sizes
            # For the puzzle piece with the biggest size, if the positions of the other puzzle image are different from the positio of this one, then we place them together once
            m = max(i for i in lst if i > 0)
            ind=lst.index(m)
            # If the first puzzle piece has the biggest size
            if(ind==0):
                # We place the first piece at its right position in the image
                back_im = Image.open(""+MySolver.path+""+main_img).copy()
                part0 = Image.open(""+MySolver.path+""+part0_img)
                back_im.paste(part0, (solx0,soly0), part0)
                lst[0]=0
                found0=True
                # If the position of puzzle piece 2 is different we place it too at its right position in the image
                if(solx0!=solx1 and soly0!=soly1):
                    if(found1==False):
                        part1 = Image.open(""+MySolver.path+""+part1_img)
                        back_im.paste(part1, (solx1,soly1), part1)
                        lst[1]=0
                        found1=True
                # If the position of puzzle piece 3 is different we place it too at its right position in the image
                if(solx0!=solx2 and soly0!=soly2):
                    if(found2==False):
                        part2 = Image.open(""+MySolver.path+""+part2_img)
                        back_im.paste(part2, (solx2,soly2), part2)
                        lst[2]=0
                        found2=True
                back_im.save(""+MySolver.path+"cut.png", quality=100)
            # If the second puzzle piece has the biggest size
            elif(ind==1):
                # We place the second piece at its right position in the image
                back_im = Image.open(""+MySolver.path+""+main_img).copy()
                part1 = Image.open(""+MySolver.path+""+part1_img)
                back_im.paste(part1, (solx1,soly1), part1)
                lst[1]=0
                found1=True
                # If the position of puzzle piece 1 is different we place it too at its right position in the image
                if(solx1!=solx0 and soly1!=soly0):
                    if(found0==False):
                        part0 = Image.open(""+MySolver.path+""+part0_img)
                        back_im.paste(part0, (solx0,soly0), part0)
                        lst[0]=0
                        found0=True
                # If the position of puzzle piece 3 is different we place it too at its right position in the image
                if(solx1!=solx2 and soly1!=soly2):
                    if(found2==False):
                        part2 = Image.open(""+MySolver.path+""+part2_img)
                        back_im.paste(part2, (solx2,soly2), part2)
                        lst[2]=0
                        found2=True
                back_im.save(""+MySolver.path+"cut.png", quality=100)
            # If the third puzzle piece has the biggest size
            else:
                # We place the third piece at its right position in the image
                back_im = Image.open(""+MySolver.path+""+main_img).copy()
                part2 = Image.open(""+MySolver.path+""+part2_img)
                back_im.paste(part2, (solx2,soly2), part2)
                lst[2]=0
                found2=True
                # If the position of puzzle piece 1 is different we place it too at its right position in the image
                if(solx2!=solx0 and soly2!=soly0):
                    if(found0==False):
                        part0 = Image.open(""+MySolver.path+""+part0_img)
                        back_im.paste(part0, (solx0,soly0), part0)
                        lst[0]=0
                        found0=True
                # If the position of puzzle piece 2 is different we place it too at its right position in the image
                if(solx2!=solx1 and soly2!=soly1):
                    if(found1==False):
                        part1 = Image.open(""+MySolver.path+""+part1_img)
                        back_im.paste(part1, (solx1,soly1), part1)
                        lst[1]=0
                        found1=True
                back_im.save(""+MySolver.path+"cut.png", quality=100)
        # We determine the coordinates of the non-white pixels on each puzzle piece image
        x0, y0, _, _=MySolver.coordinatesImg(part0_img)
        x1, y1, _, _=MySolver.coordinatesImg(part1_img)
        x2, y2, _, _=MySolver.coordinatesImg(part2_img)
        # We subtract coordinates obtained previously from the position of the puzzle pieces detected previously in order to get the final position of each piece
        X0=solx0-x0
        Y0=soly0-y0
        X1=solx1-x1
        Y1=soly1-y1
        X2=solx2-x2
        Y2=soly2-y2
        part0 = Image.open(""+MySolver.path+"part0.png")
        part1 = Image.open(""+MySolver.path+"part1.png")
        part2 = Image.open(""+MySolver.path+"part2.png")
        back_im = Image.open(""+MySolver.path+"cutcaptcha.png").copy()
        # We place each piece to its final position
        back_im.paste(part0, (X0,Y0), part0)
        back_im.paste(part1, (X1,Y1), part1)
        back_im.paste(part2, (X2,Y2), part2)
        # We save the recovered image on the disk
        back_im.save(""+MySolver.path+"captchasolved.png", quality=100)
        return X0,Y0,X1,Y1,X2,Y2

In [10]:
if __name__ == "__main__":
    root=Tk()
    app = MySolverGUI(root)
    root.mainloop()